<a href="https://colab.research.google.com/github/zeton24/gsn_iot_anomalies_detection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install polars --upgrade
! pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 35.9 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.20.2
    Uninstalling polars-0.20.2:
      Successfully uninstalled polars-0.20.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 10.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.

In [2]:
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [3]:
import os
import random
import gdown

import torch
import pytorch_lightning as pl
from torch.utils.data import random_split, DataLoader
from torchvision.transforms import Resize, Compose, ToTensor, Normalize


# DataModule

In [23]:
mqtt_train_id = '1zCd1BLTtF2JVqz-h-ydUTmpQJIjkBsNg'
intrusion_id = '1k8RqOM7hBvL8uomcKnnLr5HasYRWnYeW'

In [26]:
import polars as pol

class IoTDataset:
    def __init__(self, file_path: str, binary_classification=False):
        self.data = pol.read_csv(file_path)
        self.binary_classification = binary_classification
        if binary_classification:
          mapping = {"Anomaly": 1, "Normal": 0}
          self.labels = self.data['Label'].replace(mapping, return_dtype=pol.UInt8)
        else:
          mapping = {'Normal': 0, 'Mirai': 1, 'DoS': 2, 'Scan': 3, 'MITM ARP Spoofing': 4,
                     'MQTT_bruteforce': 5, 'Sparta': 6, 'Scan_sU': 7, 'Scan_A': 8}
          self.labels = self.data['Cat'].replace(mapping, return_dtype=pol.UInt8)
        self.data = self.data[:, :-2]
        self.length = self.__len__()

    def __len__(self):
        return self.data.select(pol.count()).item()

    def __getitem__(self, index):
        data = self.data[index].to_numpy()
        label = self.labels[index].item() if self.binary_classification else self._to_one_hot(self.labels[index], 16)
        return data, label

    def _to_one_hot(self, category, n_classes):
        result = torch.zeros(n_classes, 1)
        result[category] = 1
        return result

In [27]:
class IoTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size, file_id: str, file_name: str, num_classes: int):
        super().__init__()
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.file_id = file_id
        self.file_name = file_name

    def prepare_data(self):
      if not os.path.exists(self.file_name):
        gdown.download(id=self.file_id, output=self.file_name)

    def setup(self, stage=None):
        # train/val
        if stage == 'fit' or stage is None:
            binary = self.num_classes == 2
            dataset = IoTDataset(file_path=self.file_name, binary_classification=binary)
            train_dataset_size = int(len(dataset) * 0.9)
            self.train_dataset, self.val_dataset = random_split(dataset, [train_dataset_size, len(dataset) - train_dataset_size])
        # test
        if stage == 'test' or stage is None:
            self.test_dataset = []

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)


In [18]:
dm = IoTDataModule(batch_size=64, file_id=mqtt_train_id, file_name='mqtt_train.csv', num_classes=16)
dm.prepare_data()
dm.setup()

<ipython-input-11-e45ee8708566>:18: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  return self.data.select(pol.count()).item()


In [28]:
intrusion_dm = IoTDataModule(batch_size=64, file_id=intrusion_id, file_name='intrusion.csv', num_classes=16)
intrusion_dm.prepare_data()
intrusion_dm.setup()

<ipython-input-26-567807c92df0>:18: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  return self.data.select(pol.count()).item()


# Modele sieci 1D, 2D, 3D

In [14]:
class Model1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(32),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(64),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(128),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        # Nazwa bloku do rozważenia, taka mi się wymyśliła, ale nie upieram się przy niej.
        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 16)
            # Bez aktywacji na końcu, bo softmax się doda automatycznie razem z cross entropy.
        )

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.evaluator(x)
        return x

In [ ]:
class Model2D(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.AvgPool2d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.AvgPool2d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.AvgPool2d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),

            # Oni w artykule dają normalizację, ale pytorch wywala błąd.
            # Moim zdaniem słusznie, bo o ile rozumiem ten wzór na normalizację (co jest w dokumentacji),
            # to przy wymiarze wejścia [batch_size, 256, 1, 1] wyjście to tensor zer o takim samym wymiarze.
            # Po prostu normalizacja zmienia nam średnią na 0 (odchylenie standardowe też, ale to nieistotne),
            # a jak mamy 1 element, to zmiana średniej na 0, to zmiana elementu na 0.
            # Więc nawet jak jakoś obejdziemy ten błąd, to wyniki będą bez sensu.
            # Moim zdaniem w artykule jest błąd (a przynajmniej na rysunku, może w implementacji tego nie ma).

            # nn.BatchNorm2d(256),
            nn.Dropout(p=0.05)
        )

        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 16)
        )

    def forward(self, x):
        x = torch.reshape(x, (x.shape[0],1,8,8)) # można ten reshape dać gdzie indziej jak się znajdzie lepsze miejsce
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.evaluator(x)
        return x

In [ ]:
class Model3D(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(32),
            nn.AvgPool3d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv3d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(64),
            nn.AvgPool3d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv3d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            # nn.BatchNorm3d(128),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv3d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            # nn.BatchNorm3d(256),
            nn.Dropout(p=0.05)
        )

        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 16)
        )

    def forward(self, x):
        x = torch.reshape(x, (x.shape[0],1,4,4,4))
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.evaluator(x)
        return x

# LightningModule

In [19]:
import torchmetrics
import torchvision
from torchmetrics.classification import accuracy

class AnomalyClassifier(pl.LightningModule):

    def __init__(self, model):
        super().__init__()
        self.model = model
        self.lr = 0.001
        self.num_classes = 10
        self.current_epoch_training_loss = torch.tensor(0.0)
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=self.num_classes) # I sposób na metryke

    def forward(self, x):
        return self.model(x)

    def compute_loss(self, x, y): # model specific, x - network outputs
        return F.cross_entropy(x, y)

    def common_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = self.compute_loss(outputs,y)
        return loss, outputs, y

    def common_test_valid_step(self, batch, batch_idx):
        loss, outputs, y = self.common_step(batch, batch_idx)
        preds = torch.argmax(outputs, dim=1)
        acc = torchmetrics.functional.accuracy(preds, y, num_classes = self.num_classes, task="multiclass") # II sposób na metryke
        return loss, acc

    def training_step(self, batch, batch_idx):
        loss, outputs, y = self.common_step(batch, batch_idx)
        self.training_step_outputs.append(loss)
        accuracy = self.accuracy(outputs, y) # I sposób na metryke
        self.log_dict( # I sposób logowania
            {
                "train_loss": loss,
                "train_accuracy": accuracy
            },
            on_step = False,
            on_epoch = True,
            prog_bar = True
        )

        return {'loss':loss}

    def on_train_epoch_end(self):
        outs = torch.stack(self.training_step_outputs)
        self.current_epoch_training_loss = outs.mean()
        self.training_step_outputs.clear()

    def validation_step(self, batch, batch_idx):
        loss, acc = self.common_test_valid_step(batch, batch_idx)
        self.validation_step_outputs.append(loss)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {'val_loss':loss, 'val_acc': acc}

    def on_validation_epoch_end(self):
        outs = torch.stack(self.validation_step_outputs)
        avg_loss = outs.mean()
        self.logger.experiment.add_scalars('train and vall losses', {'train': self.current_epoch_training_loss.item() , 'val': avg_loss.item()}, self.current_epoch)
        self.validation_step_outputs.clear()

    def test_step(self, batch, batch_idx):
        loss, acc = self.common_test_valid_step(batch, batch_idx)
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True) # II sposób logowania
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {'test_loss':loss, 'test_acc': acc} ##

    def configure_optimizers(self):
        optimizer =  torch.optim.Adam(self.parameters(), lr=self.lr)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
        return [optimizer], [lr_scheduler]

In [29]:
trainer = pl.Trainer(accelerator="auto", max_epochs=10)
model = Model1D()
classifier = AnomalyClassifier(model=model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [30]:
trainer.fit(classifier, intrusion_dm)

<ipython-input-26-567807c92df0>:18: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  return self.data.select(pol.count()).item()
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type               | Params
------------------------------------------------
0 | model    | Model1D            | 749 K 
1 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
749 K     Trainable params
0         Non-trainable params
749 K     Total params
2.999     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: expected scalar type Double but found Float

## Elementy do wykorzystania

### Batch loader

In [ ]:
# Na bazie datasetu z labów - zwraca kolejne batche.
# Nie jest jakiś genialny, ale i tak go używam tylko tu do testów.
class Dataset:
  def __init__(self, data, labels, batch_size=1):
    self.data = data
    self.labels = labels
    self.batch_size = batch_size
    self.n_batches = len(self.data) // batch_size

  def __iter__(self):
    for i in range(self.n_batches):
      inputs = torch.zeros(self.batch_size, 1, 64) # shape zahardkodowany
      labels = torch.zeros(self.batch_size, self.labels.shape[1])

      for j in range(self.batch_size):
        inputs[j, 0] = self.data[i*self.batch_size+j]
        labels[j]= self.labels[i*self.batch_size+j]

      yield inputs, labels
  # todo - jakiś shuffle by się przydał, ale to tylko jeśli byśmy z tego korzystały

### Funkcje do treningu i testowania

In [ ]:
# Funkcja przeprowadzająca traning

def train(net, loader, optimizer, criterion, epochs):
  for epoch in range(epochs):
    epoch_loss = 0.
    for i, (inputs, labels) in enumerate(loader):

      # zero the parameter gradients
      optimizer.zero_grad()

      outputs = net(inputs)
      loss = criterion(outputs, labels)

      loss.backward()
      optimizer.step()

      epoch_loss += loss
    if epoch % 5 == 4: # wypisywanie co 5 epok
      print(f'[{epoch + 1}, {i + 1:5d}] eloss: {epoch_loss:.3f}')

In [ ]:
def test(network, test_set, n_classes = 16):
  network.eval() # Przełącza sieć w tryb testowania, m. in. wyłącza dropouta.
  err_matrix = torch.zeros((n_classes, n_classes), dtype=int)

  with torch.no_grad():

    l = 0 # Obliczam koszt, żeby wiedzieć, co się dzieje, docelowo tego nie będzie.
    for inputs, labels in test_set:

      outputs = network(inputs)
      l += criterion(outputs, labels)

      _, label = torch.max(labels, 1)
      _, predicted = torch.max(outputs, 1)

      for truth, prediction in zip(label, predicted):
        err_matrix[truth, prediction] += 1

  network.train() # Przełącza z powrotem w tryb treningu.
  return err_matrix, l

In [ ]:
# Funkcja tworzy losowe etykiety one-hot.
def rand_and_predict(dataset_size, output_length):
  labels = torch.zeros(dataset_size, output_length)
  for i in range(dataset_size):
    hot = torch.randint(output_length,(1,))
    labels[i, hot] = 1
  return labels

## Test na losowych danych

In [ ]:
inputs = torch.rand(256,1,64)
labels = rand_and_predict(256,16)

In [ ]:
loader = Dataset(inputs, labels, batch_size=32)
criterion = nn.CrossEntropyLoss()
criterion_test = nn.CrossEntropyLoss()

### Model 1D

In [ ]:
network1 = Model1D()
optimizer1 = optim.Adam(network1.parameters(), lr=0.001)

In [ ]:
train(network1, loader, optimizer1, criterion, 10)

[5,     8] eloss: 0.968
[10,     8] eloss: 0.038


In [ ]:
result1, loss = test(network1, loader)
print(f"Accuracy: {sum(sum(result1*torch.eye(16)))*100/sum(sum(result1)):.2f}%")
print(loss)
print(result1)

Accuracy: 100.00%
tensor(0.0329)
tensor([[ 9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0, 21,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 20,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0],
     

### Model 2D

In [ ]:
network2 = Model2D()
optimizer2 = optim.Adam(network2.parameters(), lr=0.001)

In [ ]:
train(network2, loader, optimizer2, criterion, 20)

[5,     8] eloss: 15.003
[10,     8] eloss: 1.362
[15,     8] eloss: 0.934
[20,     8] eloss: 0.649


In [ ]:
result2, loss = test(network2, Dataset(inputs, labels))
print(f"Accuracy: {sum(sum(result2*torch.eye(16)))*100/sum(sum(result2)):.2f}%")
print(loss)
print(result2)

Accuracy: 98.44%
tensor(14.8871)
tensor([[ 9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 15,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0],
        [ 0,  0,  0,  0,  0, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0, 21,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 20,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0],
     

### Model 3D

In [ ]:
network3 = Model3D()
optimizer3 = optim.Adam(network3.parameters(), lr=0.001)

In [ ]:
train(network3, loader, optimizer3, criterion, 30)

[5,     8] eloss: 20.926
[10,     8] eloss: 18.663
[15,     8] eloss: 15.994
[20,     8] eloss: 12.420
[25,     8] eloss: 9.254
[30,     8] eloss: 6.468


In [ ]:
result3, loss = test(network3, Dataset(inputs, labels))
print(f"Accuracy: {sum(sum(result3*torch.eye(16)))*100/sum(sum(result3)):.2f}%")
print(loss)
print(result3)

Accuracy: 68.75%
tensor(215.7659)
tensor([[ 1,  0,  0,  0,  0,  0,  2,  0,  0,  0,  6,  0,  0,  0,  0,  0],
        [ 0, 18,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0],
        [ 0,  0, 14,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0],
        [ 0,  2,  0, 16,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 10,  0,  0,  1,  0,  0,  1,  0,  0,  5,  0,  0],
        [ 0,  0,  0,  0,  2, 16,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0, 11,  0,  0,  0,  2,  0,  0,  0,  0,  0],
        [ 0,  0,  1,  0,  1,  2,  1,  8,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1,  0,  0,  2,  1,  0,  1, 12,  0,  0,  0,  0,  4,  0,  0],
        [ 1,  2,  1,  0,  1,  1,  0,  1,  0, 10,  1,  0,  0,  0,  1,  1],
        [ 0,  0,  1,  0,  1,  0,  3,  0,  0,  0,  8,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  1,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  5,  0,  0,  0,  0,  0],
    

## Notatki, żeby nie szukać ciągle w artykule, bałagan trochę, ale są

koszt - cross entropy

aktywacja lineara - relu

aktywacja po 16 neuronach - softmax

zrobili L1, L2 i dropout, ale nie wchodziłam w to dokładnie

batch - 64, 128 dawały najlepsze wyniki

eksperymentalnie sprawdzili, że 100 epok daje zbieżność

w adamie dali 0.0001 learning rate
